In [1]:
import numpy as np
import pandas as pd
import os
from datetime import datetime
import pickle
# import matplotlib.pyplot as plt
from time import time
import sys

In [2]:
PATH_TO_REPO = '/n/dtak/mimic-pipeline-tools/'

sys.path.append(PATH_TO_REPO+'mimic-iii-v1-4/extract-scripts')
from data_load_utils import load_labs_and_vitals,convert_1inds_to_everinds,convert_1inds_to_kinds

PATH_TO_QUERY_DATA = PATH_TO_REPO+"mimic-iii-v1-4/query-data/"
PATH_TO_CLEANED_DATA = "/n/home07/carissawu/intervention-onset/model-data/"

pd.set_option("display.max_columns",200)

###########
#params for dataset construction; make same as UT

LOS_LOWER_THRESH = 6 #exclude if LOS is shorter than this many hours
LOS_UPPER_THRESH = 400 #exclude if LOS is longer than this many hours

CONTROL_ENDTIME_UPPERQUANT = 0.9 #quantile for upper end of when we'll choose control time per enc; -1=use all, no cut
DISC_GRID_SIZE = 1.0 #number of hours of separation in grid
INTERVENTION_TYPE = 'vasopressor' #for now just vasopressor, later other types, too...

#NOTE: THIS PARAM IS REALLY, REALLY IMPT! DEFINITELY PLOT *WHEN* THE FIRST TIME 
# EACH INTERVENTION IS GIVEN TO DECIDE WHAT TO SET THIS!!
INTERV_START_THRESH = 6

SEED = 8675309
np.random.seed(SEED)

START_TIME = 0 #start time series here...

##########
########## load in the baseline/static data for the cohort
##########

cohort_dat = pd.read_csv(PATH_TO_QUERY_DATA+'cohort.csv')
og_row_num = cohort_dat.shape[0]

In [3]:

# do any additional filtering beyond cohort construction, if desired...

cohort_dat = cohort_dat.loc[cohort_dat['LOS'] >= LOS_LOWER_THRESH,:]
cohort_dat = cohort_dat.loc[cohort_dat['LOS'] <=  LOS_UPPER_THRESH,:]

cohort_dat['INTIME'] = pd.to_datetime(cohort_dat['INTIME'])

print('cohort loaded')

cohort loaded


In [4]:
new_row_num = cohort_dat.shape[0]

In [5]:
og_row_num

21583

In [6]:
new_row_num

20969

In [7]:
new_row_num / og_row_num

0.9715516841958949

In [8]:
cohort_dat['HOSPITAL_EXPIRE_FLAG'].value_counts()

0    10497
1      743
Name: HOSPITAL_EXPIRE_FLAG, dtype: int64

In [10]:
# does HOSPITAL_EXPIRE_FLAG equal 1 when they die, or if they die?

In [12]:
one_ind = cohort_dat.index[cohort_dat['HOSPITAL_EXPIRE_FLAG'] == 1].tolist()

In [8]:
from os import listdir
from os.path import isfile, join

In [9]:
##########
########## load in pressor data, or other intervention data
##########     TODO: add other loaders here for other interventions of interest here...
##########

# other interventions will have different processing here, eg for fluids...
if INTERVENTION_TYPE == 'vasopressor':
    interven_dat = pd.read_csv(PATH_TO_QUERY_DATA+"vasopressors_mv.csv")
    interven_dat['STARTTIME'] = pd.to_datetime(interven_dat['STARTTIME'])
    # vaso_dat = vaso_dat.loc[vaso_dat['ITEMID']!=222315,:] #exclude vasopressin...?
    interven_dat = interven_dat.loc[:,['ICUSTAY_ID','STARTTIME']]

    #get the first time a vaso is given
    interven_dat = interven_dat.groupby(['ICUSTAY_ID']).min()
    interven_dat.reset_index(level=0, inplace=True)
    print(interven_dat)
    
# other interventions will have different processing here, eg for fluids...
if INTERVENTION_TYPE == 'fluids':
    interven_dat = pd.read_csv(PATH_TO_QUERY_DATA+"allfluids_and_bloodproducts_mv.csv")
    interven_dat['STARTTIME'] = pd.to_datetime(interven_dat['STARTTIME'])
    # vaso_dat = vaso_dat.loc[vaso_dat['ITEMID']!=222315,:] #exclude vasopressin...?
    interven_dat = interven_dat.loc[:,['ICUSTAY_ID','STARTTIME']]

    #get the first time a vaso is given
    interven_dat = interven_dat.groupby(['ICUSTAY_ID']).min()
    interven_dat.reset_index(level=0, inplace=True)
    
if INTERVENTION_TYPE == 'death':
    # only for ICUSTAY_IDs where the flag is 1, include the starttime.
    # if HOSPITAL_EXPIRE_FLAG == 1, then create a new dataframe which stores the DEATHTIME.

    death_dict = {}
    death_dict['ICUSTAY_ID'] = []
    death_dict['STARTTIME'] = []

    for name, group in cohort_dat.groupby(['ICUSTAY_ID']):
        # calculate the mean hospital expire flag of the group
        if np.mean(group['HOSPITAL_EXPIRE_FLAG']) > 0:
            # add to dictionary
            death_dict['ICUSTAY_ID'].append(name)
            death_dict['STARTTIME'].append(pd.to_datetime(group.iloc[0]['DEATHTIME']))

    interven_dat = pd.DataFrame(death_dict)
    interven_dat.reset_index(level=0, inplace=True)
    print(interven_dat)

#NOTE: after processing, interven_dat should have 2 cols: ID & Start_time for those who got it 
#interven_dat should now just have 2 cols:

cohort_dat = cohort_dat.merge(interven_dat,'left','ICUSTAY_ID')
cohort_dat['Interv_Time_After_Start'] = (cohort_dat['STARTTIME']-cohort_dat['INTIME']).dt.total_seconds()/60/60

#logic for filtering out bad interventions, ie too early or too late (after LOS thresh)
keep_inds = np.logical_or(
    np.logical_and(cohort_dat['Interv_Time_After_Start']>=INTERV_START_THRESH,
        cohort_dat['Interv_Time_After_Start']<=LOS_UPPER_THRESH),
    cohort_dat['Interv_Time_After_Start'].isnull())
print(len(keep_inds))

cohort_dat = cohort_dat.loc[keep_inds,:]
cohort_dat.reset_index(inplace=True)
cohort_dat = cohort_dat.sort_values(by=["ICUSTAY_ID"])

print('%d / %d () patients with intervention %s' %(
    cohort_dat['Interv_Time_After_Start'].notnull().sum(),cohort_dat.shape[0],INTERVENTION_TYPE))

final_ICU_IDs = np.array(cohort_dat['ICUSTAY_ID'])

      ICUSTAY_ID           STARTTIME
0         200024 2127-03-03 16:15:00
1         200028 2133-10-29 17:47:00
2         200033 2198-08-10 18:30:00
3         200034 2186-01-21 12:38:00
4         200063 2141-03-21 11:00:00
5         200075 2159-09-23 01:40:00
6         200087 2196-08-31 08:14:00
7         200095 2113-10-30 23:30:00
8         200116 2198-03-19 22:23:00
9         200131 2176-10-30 13:37:00
10        200141 2143-10-15 14:19:00
11        200143 2191-04-03 10:02:00
12        200172 2198-05-18 04:15:00
13        200183 2191-06-09 07:01:00
14        200208 2117-02-18 20:50:00
15        200223 2159-01-18 17:55:00
16        200231 2127-03-09 16:30:00
17        200238 2117-04-24 05:45:00
18        200282 2164-05-04 23:18:00
19        200286 2192-01-03 23:03:00
20        200290 2102-12-24 19:01:00
21        200346 2118-12-05 18:05:00
22        200347 2116-06-07 16:00:00
23        200349 2139-06-01 19:05:00
24        200392 2113-02-10 14:08:00
25        200441 2166-08-23 18:21:00
2

In [10]:
# Print IDs that have the intervention

cohort_dat['Interv_Time_After_Start'].notnull()


0        False
1        False
2        False
3        False
4        False
5         True
6         True
7        False
8        False
9        False
10       False
11       False
12       False
13       False
14       False
15       False
16       False
17        True
18       False
19        True
20       False
21       False
22       False
23       False
24       False
25       False
26       False
27       False
28       False
29       False
         ...  
16676    False
16677    False
16678    False
16679     True
16680    False
16681    False
16682    False
16683    False
16684    False
16685    False
16686    False
16687    False
16688    False
16689    False
16690    False
16691     True
16692    False
16693    False
16694    False
16695    False
16696    False
16697    False
16698    False
16699    False
16700    False
16701    False
16702     True
16703    False
16704    False
16705    False
Name: Interv_Time_After_Start, Length: 16706, dtype: bool

In [11]:
final_ICU_IDs.shape

(16706,)

In [12]:

##############
############## load in all the ts data
############## 

all_ts_dats, VAR_REFERENCE_IMPUTE, all_ts_vars = load_labs_and_vitals(PATH_TO_QUERY_DATA,cohort_dat)

# it is a time-series, so I may need to modify load_labs_and_vitals


##############
############## finally build out dataset by discretizing time
############## 

#store final dataset after right alignment here...
all_grid_data = {}

#precompute
starts_ts = {}
ends_ts = {}
for v in all_ts_dats.keys():
    starts_ts[v] = all_ts_dats[v]['ICUSTAY_ID'].searchsorted(final_ICU_IDs,'left')
    ends_ts[v] = all_ts_dats[v]['ICUSTAY_ID'].searchsorted(final_ICU_IDs,'right')


dbp
fio2


/n/helmod/apps/centos7/Core/Anaconda3/5.0.1-fasrc01/x/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2802: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


GCS
hr
map
sbp
spontaneousrr


/n/helmod/apps/centos7/Core/Anaconda3/5.0.1-fasrc01/x/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2802: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


spo2
temp
urine
weight
bun


/n/helmod/apps/centos7/Core/Anaconda3/5.0.1-fasrc01/x/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2802: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


magnesium
platelets
sodium
alt
hct
po2
ast
potassium
wbc
bicarbonate
creatinine
lactate
pco2
bilirubin_total
glucose


/n/helmod/apps/centos7/Core/Anaconda3/5.0.1-fasrc01/x/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2802: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


inr
hgb
time series variables all loaded!
time series variables clipped to plausible ranges, outliers removed


In [13]:
loop_t = time()
num_excluded = 0
for ID_ind,ID in enumerate(final_ICU_IDs):
    if ID_ind % 100==99:
        print("processing %d/%d, took %.2f" %(ID_ind+1,len(final_ICU_IDs),time()-loop_t))
        loop_t = time()

    this_static_dat = cohort_dat.iloc[ID_ind]

    #TODO: you may want to add other vars in besides just age...
    # print(this_static_dat.keys())
    
    age = this_static_dat['AGE']
    gender = this_static_dat['GENDER'] == 'F'
    adm_type = this_static_dat['ADMISSION_TYPE']
    
    is_urgent = adm_type == 'URGENT'
    is_emergency = adm_type == 'EMERGENCY'
    
    is_sicu = this_static_dat['FIRST_CAREUNIT'] == 'SICU'
    is_tsicu = this_static_dat['FIRST_CAREUNIT'] == 'TSICU'
    is_micu = this_static_dat['FIRST_CAREUNIT'] == 'MICU'
    is_csru = this_static_dat['FIRST_CAREUNIT'] == 'CSRU'
    
    if age > 90: #fix ages set to 300+ for anonymity purposes
        age = 90

    assert np.logical_not(np.isnan(age))
    start_time = this_static_dat['INTIME']

    
    if INTERVENTION_TYPE == 'death':
        # for this task, end_time is not selected randomly; include entire trajectory
        if pd.isnull(this_static_dat['Interv_Time_After_Start']):
            end_time = this_static_dat['LOS']
            has_interv = False
        else:
            end_time = this_static_dat['Interv_Time_After_Start']
            has_interv = True
    else:
        #end time should be a random time between VASO_START_THRESH hours in and 90% quantile of LOS
        if pd.isnull(this_static_dat['Interv_Time_After_Start']):
            if CONTROL_ENDTIME_UPPERQUANT==-1: #just use true end time, nothing shorter
                end_time = this_static_dat['LOS']
            else:
                end_time = np.random.uniform(LOS_LOWER_THRESH,CONTROL_ENDTIME_UPPERQUANT*this_static_dat['LOS'],1)[0]
            has_interv = False
        else:
            end_time = this_static_dat['Interv_Time_After_Start']
            has_interv = True

    # #times at which to build out feature vectors
    # if CONTROL_ENDTIME_UPPERQUANT==-1: #just use true end time, nothing shorter
    #     #forwards, left align from start
    #     grid_times = np.arange(START_TIME,end_time,DISC_GRID_SIZE)
    # else:
    #     #backwards, right align from end
    #     grid_times = np.arange(end_time,0,-DISC_GRID_SIZE)[::-1]

    #always right align so not starting right at 0; get a little data
    grid_times = np.arange(end_time,START_TIME,-DISC_GRID_SIZE)[::-1]
#     print("end time")
#     print(end_time)
#     print("START_TIME")
#     print(START_TIME)
#     print("grid times length")
#     print(len(grid_times))

    n_t = len(grid_times)
    #print('Original timeseries time: ' + str(n_t))

    this_grid_dat = pd.DataFrame()
    this_grid_dat['Times'] = grid_times 
    this_grid_dat['Has_Interv'] = has_interv
    this_grid_dat['Interv_Time'] = this_static_dat['Interv_Time_After_Start']
    this_grid_dat['Age'] = age
    this_grid_dat['is_F'] = gender
    this_grid_dat['is_urgent'] = is_urgent
    this_grid_dat['is_emergency'] = is_emergency
    this_grid_dat['is_sicu'] = is_sicu
    this_grid_dat['is_tsicu'] = is_tsicu
    this_grid_dat['is_micu'] = is_micu
    this_grid_dat['is_csru'] = is_csru
    
    # Treat first-measured weight as static.
    # If the patient never has their weight taken, exclude them.

    #####
    #now build out the time series
    #also build out indicator vector at each time, noting whether var was imputed or not
    for v in all_ts_vars:
        dat = all_ts_dats[v]

        s = starts_ts[v][ID_ind]
        e = ends_ts[v][ID_ind]
        this_dat = dat[s:e]
        this_t = np.array((this_dat['CHARTTIME'] - start_time).astype('timedelta64[m]').astype(float))/60

        #urine has no valuenum field
        if v=='urine':
            val_str = 'VALUE'
        else:
            val_str = 'VALUENUM'

        this_vals = np.array(this_dat[val_str],"float")

        #impute anything initially missing with pop median, then
        #fill this in with observed values via LOCF
        imputed_vals = VAR_REFERENCE_IMPUTE[v]*np.ones(n_t)

        ### Now get LOCF for cts labs/vitals
        tt = np.searchsorted(grid_times+1e-8,this_t)
        
        for i in range(len(tt)):
            if i!=len(tt)-1:
                imputed_vals[tt[i]:tt[i+1]] = this_vals[i] 
            else:
                imputed_vals[tt[i]:] = this_vals[i] 
        
        this_grid_dat[v] = imputed_vals

        #get indicators for at which times the variable was actually sampled
        inds_samples_vals = np.zeros(n_t+1) #edge case when values past endtime
        inds_samples_vals[tt] = 1.0
        inds_samples_vals = inds_samples_vals[:-1] #edge case when values past endtime
        
        if v == 'weight':
            # Append first time the weight was measured
            weight_meas_times = np.where(inds_samples_vals == 1)[0]
            if len(weight_meas_times > 0):
                first_weight_time = weight_meas_times[0]
                
                # if possible, add 1 to account for the case where the weight isn't recorded until after the measurement time
                if first_weight_time < (len(imputed_vals) - 1):
                    first_weight_time += 1
                this_grid_dat['Weight'] = imputed_vals[first_weight_time]

            else:
                num_excluded += 1
            
        else:

            this_grid_dat[v+'_ind'] = inds_samples_vals

            # extra inds for other times...
            if v in ['ADBP','AMBP','ASBP','Lactate']:
                ever_inds = convert_1inds_to_everinds(inds_samples_vals)
                this_grid_dat[v+'_ever-ind'] = ever_inds

            # extra inds for other times...
            if v in ['ALT','AST','BUN','Bilirubin','Creatinine','Glucose','Hct','Hgb','FiO2',
                'HCO3','PCO2','PO2','K','Na','Lactate','Magnesium','Platelet','WBC','Weight']:
                last_8_inds = convert_1inds_to_kinds(inds_samples_vals,8)
                this_grid_dat[v+'_8-ind'] = last_8_inds


    #finished all vars; now combine the 3 GCS vars together and drop old ones
    this_grid_dat['GCS'] = this_grid_dat['GCS_eye']+this_grid_dat['GCS_motor']+this_grid_dat['GCS_verbal']

    this_grid_dat['GCS_ind'] = this_grid_dat['GCS_eye_ind']+this_grid_dat['GCS_motor_ind']+this_grid_dat['GCS_verbal_ind']
    this_grid_dat['GCS_ind'] = np.minimum(this_grid_dat['GCS_ind'],1)
    
    #print('Length of this_grid_dat: ' + str(this_grid_dat['GCS'].shape))
    
    if len(weight_meas_times > 0):
        all_grid_data[ID] = this_grid_dat


processing 100/16706, took 3.97
processing 200/16706, took 3.95
processing 300/16706, took 4.00
processing 400/16706, took 4.02
processing 500/16706, took 3.96
processing 600/16706, took 4.07
processing 700/16706, took 3.97
processing 800/16706, took 3.96
processing 900/16706, took 4.06
processing 1000/16706, took 3.97
processing 1100/16706, took 3.96
processing 1200/16706, took 4.07
processing 1300/16706, took 4.02
processing 1400/16706, took 3.99
processing 1500/16706, took 3.97
processing 1600/16706, took 4.08
processing 1700/16706, took 3.96
processing 1800/16706, took 3.96
processing 1900/16706, took 4.07
processing 2000/16706, took 4.13
processing 2100/16706, took 3.97
processing 2200/16706, took 3.98
processing 2300/16706, took 4.01
processing 2400/16706, took 3.97
processing 2500/16706, took 4.05
processing 2600/16706, took 4.20
processing 2700/16706, took 3.98
processing 2800/16706, took 3.97
processing 2900/16706, took 3.96
processing 3000/16706, took 3.97
processing 3100/167

KeyboardInterrupt: 

In [14]:
# num_excluded
this_grid_dat['Times']

0      0.83813
1      1.83813
2      2.83813
3      3.83813
4      4.83813
5      5.83813
6      6.83813
7      7.83813
8      8.83813
9      9.83813
10    10.83813
11    11.83813
12    12.83813
13    13.83813
14    14.83813
15    15.83813
16    16.83813
17    17.83813
18    18.83813
19    19.83813
20    20.83813
21    21.83813
22    22.83813
23    23.83813
24    24.83813
25    25.83813
26    26.83813
27    27.83813
28    28.83813
29    29.83813
30    30.83813
31    31.83813
32    32.83813
Name: Times, dtype: float64

In [ ]:
OUTPUT_PATH = PATH_TO_CLEANED_DATA+'11-15-21-interpretable-timeseries-vasopressor.p'

print(OUTPUT_PATH)
pickle.dump(all_grid_data,open(OUTPUT_PATH,'wb'))
pickle.dump(all_grid_data,open('data.p', 'wb'))

print('dumped')

/n/home07/carissawu/intervention-onset/model-data/11-15-21-interpretable-timeseries-vasopressor.p


# Replicating Previous Vasopressor Work

In [6]:
PATH_TO_REPO = '/n/dtak/mimic-pipeline-tools/'

sys.path.append(PATH_TO_REPO+'mimic-iii-v1-4/extract-scripts')
from data_load_utils import load_labs_and_vitals,convert_1inds_to_everinds,convert_1inds_to_kinds

PATH_TO_QUERY_DATA = PATH_TO_REPO+"mimic-iii-v1-4/query-data/"
PATH_TO_CLEANED_DATA = "/n/home07/carissawu/intervention-onset/model-data/"

pd.set_option("display.max_columns",200)

###########
#params for dataset construction; make same as UT

LOS_LOWER_THRESH = 6 #exclude if LOS is shorter than this many hours
LOS_UPPER_THRESH = 600 #exclude if LOS is longer than this many hours

CONTROL_ENDTIME_UPPERQUANT = 0.9 #quantile for upper end of when we'll choose control time per enc; -1=use all, no cut
DISC_GRID_SIZE = 1.0 #number of hours of separation in grid
INTERVENTION_TYPE = 'vasopressor' #for now just vasopressor, later other types, too...

#NOTE: THIS PARAM IS REALLY, REALLY IMPT! DEFINITELY PLOT *WHEN* THE FIRST TIME 
# EACH INTERVENTION IS GIVEN TO DECIDE WHAT TO SET THIS!!
INTERV_START_THRESH = 6

SEED = 8675309
np.random.seed(SEED)

START_TIME = 0 #start time series here...

##########
########## load in the baseline/static data for the cohort
##########

cohort_dat = pd.read_csv(PATH_TO_QUERY_DATA+'cohort.csv')
og_row_num = cohort_dat.shape[0]

In [7]:

# do any additional filtering beyond cohort construction, if desired...

cohort_dat = cohort_dat.loc[cohort_dat['LOS'] >= LOS_LOWER_THRESH,:]
cohort_dat = cohort_dat.loc[cohort_dat['LOS'] <=  LOS_UPPER_THRESH,:]

cohort_dat['INTIME'] = pd.to_datetime(cohort_dat['INTIME'])

print('cohort loaded')

cohort loaded


In [8]:
interven_dat = pd.read_csv(PATH_TO_QUERY_DATA+"vasopressors_mv.csv")
interven_dat['STARTTIME'] = pd.to_datetime(interven_dat['STARTTIME'])
interven_dat = interven_dat.loc[:,['ICUSTAY_ID','STARTTIME']]
interven_dat = interven_dat.groupby(['ICUSTAY_ID']).min()
interven_dat.reset_index(level=0, inplace=True)
cohort_dat = cohort_dat.merge(interven_dat,'left','ICUSTAY_ID')
cohort_dat['Interv_Time_After_Start'] = (cohort_dat['STARTTIME']-cohort_dat['INTIME']).dt.total_seconds()/60/60
keep_inds = np.logical_or(
    np.logical_and(cohort_dat['Interv_Time_After_Start']>=INTERV_START_THRESH,
        cohort_dat['Interv_Time_After_Start']<=LOS_UPPER_THRESH),
    cohort_dat['Interv_Time_After_Start'].isnull())
print(len(keep_inds))
print("cohort shape")
print(cohort_dat.shape[0])
cohort_dat = cohort_dat.loc[keep_inds,:]
cohort_dat.reset_index(inplace=True)
cohort_dat = cohort_dat.sort_values(by=["ICUSTAY_ID"])
print("cohort shape")
print(cohort_dat.shape[0])
print('%d / %d () patients with intervention %s' %(
    cohort_dat['Interv_Time_After_Start'].notnull().sum(),cohort_dat.shape[0],INTERVENTION_TYPE))

# cohort_dat

20510
cohort shape
20510
cohort shape
16401
1473 / 16401 () patients with intervention vasopressor


In [124]:
cohort_1 = cohort_dat[cohort_dat['Interv_Time_After_Start'].notnull()]
cohort_0 = cohort_dat[cohort_dat['Interv_Time_After_Start'].isnull()]

In [129]:
cohort_1_age = np.array(cohort_1['AGE'])
print(np.median(cohort_1_age))
cohort_0_age = np.array(cohort_0['AGE'])
print(np.median(cohort_0_age))

67.3621503758
64.3217021472
